In [1]:
import os
import pickle
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from bvh_converter import bvh_mod
from scipy.signal import savgol_filter

In [ ]:
file_name = "BKO_E1_D1_07_Suku"
pickle_path = f'data/motion_data_pkl/{file_name}_T.pkl'
    
# if os.path.isfile(pickle_path):
with open(pickle_path, 'rb') as file:
    motiondata_section = pickle.load(file)
print(f"Loaded {file_name} pickle")
    
Lfoot_xyzpos_data = motiondata_section["position"]["SEGMENT_LEFT_FOOT"]
Lfoot_zpos_data = savgol_filter(Lfoot_xyzpos_data[:,2], 60, 0)

Rfoot_xyzpos_data = motiondata_section["position"]["SEGMENT_RIGHT_FOOT"]
Rfoot_zpos_data = savgol_filter(Rfoot_xyzpos_data[:,2], 60, 0)

# V4

In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D
from scipy.interpolate import interp1d

def plot_foot_trajectories(
    file_name: str,
    mode: str,
    base_path_cycles: str = "data/virtual_cycles",
    base_path_logs: str = "data/logs_v1_may",
    frame_rate: float = 240,
    W_start: float = 170.0,
    W_end: float = 185.0,
    n_beats_per_cycle: int = 4,
    n_subdiv_per_beat: int = 12,
    nn: int = 8,
    figsize: tuple = (12, 6),
    dpi: int = 300
):
    """
    Plot left- and right-foot Y-position trajectories ±window around each downbeat,
    marking foot-onset times for cycles that have an onset in the window.

    Parameters
    ----------
    file_name
        Base name (e.g. "BKO_E1_D2_03_Suku")
    base_path_cycles
        Directory containing your virtual_cycles CSVs
    base_path_logs
        Directory containing your logs_v1_may/<file>_T/onset_info
    frame_rate
        Frame rate of the motion capture data
    W_start, W_end
        Time window (in seconds) around which to plot trajectories
    n_beats_per_cycle
        Beats per cycle (e.g. 4)
    n_subdiv_per_beat
        Subdivisions per beat (e.g. 12)
    nn
        Half-width in subdivisions: window_size_subdiv = 2*nn
    figsize
        Matplotlib figure size tuple
    dpi
        Matplotlib figure DPI

    Returns
    -------
    fig, ax
        The matplotlib Figure and Axes objects
    """
    # build file paths
    cycles_csv = os.path.join(base_path_cycles, f"{file_name}_C.csv")
    logs_onset_dir = os.path.join(base_path_logs, f"{file_name}_T", "onset_info")
    left_onsets_csv  = os.path.join(logs_onset_dir, f"{file_name}_T_left_foot_onsets.csv")
    right_onsets_csv = os.path.join(logs_onset_dir, f"{file_name}_T_right_foot_onsets.csv")
    left_zpos_csv    = os.path.join(logs_onset_dir, f"{file_name}_T_left_foot_zpos.csv")
    right_zpos_csv   = os.path.join(logs_onset_dir, f"{file_name}_T_right_foot_zpos.csv")

    # load data
    Lz = pd.read_csv(left_zpos_csv)["zpos"].values
    Rz = pd.read_csv(right_zpos_csv)["zpos"].values
    n_frames = len(Lz)
    times = np.arange(n_frames) / frame_rate

    # interpolation functions
    L_interp = interp1d(times, Lz, bounds_error=False, fill_value="extrapolate")
    R_interp = interp1d(times, Rz, bounds_error=False, fill_value="extrapolate")

    # trim to window
    win_mask = (times >= W_start) & (times <= W_end)
    t_win = times[win_mask]
    L_win = Lz[win_mask]
    R_win = Rz[win_mask]

    # cycles (downbeats)
    cyc_df = pd.read_csv(cycles_csv)
    cyc_df = cyc_df[(cyc_df["Virtual Onset"] >= W_start) & (cyc_df["Virtual Onset"] <= W_end)]
    onsets = cyc_df["Virtual Onset"].values[:-1]
    durations = np.diff(cyc_df["Virtual Onset"].values)
    avg_cycle = durations.mean()

    # foot onsets
    left_df  = pd.read_csv(left_onsets_csv)
    right_df = pd.read_csv(right_onsets_csv)
    left_times  = left_df[ (left_df["time_sec"]>=W_start)&(left_df["time_sec"]<=W_end) ]["time_sec"].values
    right_times = right_df[(right_df["time_sec"]>=W_start)&(right_df["time_sec"]<=W_end)]["time_sec"].values

    # window half-width in seconds
    beat_len   = avg_cycle / n_beats_per_cycle
    subdiv_len = beat_len / n_subdiv_per_beat
    half_win   = subdiv_len * nn

    # collect cycles that have foot onsets
    cyc_L, L_near = [], {}
    for c in onsets:
        hits = left_times[(left_times>=c-half_win)&(left_times<=c+half_win)]
        if len(hits):
            cyc_L.append(c); L_near[c] = hits

    cyc_R, R_near = [], {}
    for c in onsets:
        hits = right_times[(right_times>=c-half_win)&(right_times<=c+half_win)]
        if len(hits):
            cyc_R.append(c); R_near[c] = hits

    # plotting
    fig, ax = plt.subplots(figsize=figsize, dpi=dpi)
    cmap = plt.get_cmap('cool')
    t_range = W_end - W_start

    # left foot
    for i, c in enumerate(cyc_L):
        col = cmap((c-W_start)/t_range)
        m = (t_win>=c-half_win)&(t_win<=c+half_win)
        tr = t_win[m] - c
        ax.plot(tr, L_win[m], '-', color=col, alpha=0.3,
                label="Left Foot" if i==0 else "")
        for lt in L_near[c]:
            rel = lt - c
            ax.axvline(rel, color=col, linestyle='-', alpha=0.5)
            ax.plot(rel, L_interp(lt), 'o', ms=8, markeredgecolor='k', alpha=0.8)

    # right foot
    for i, c in enumerate(cyc_R):
        col = cmap((c-W_start)/t_range)
        m = (t_win>=c-half_win)&(t_win<=c+half_win)
        tr = t_win[m] - c
        ax.plot(tr, R_win[m], '--', color=col, alpha=0.3,
                label="Right Foot" if i==0 else "")
        for rt in R_near[c]:
            rel = rt - c
            ax.axvline(rel, color=col, linestyle='--', alpha=0.5)
            ax.plot(rel, R_interp(rt), 'x', ms=8, markeredgecolor='k', alpha=0.8)

    # decorations
    ax.axvline(0, color='k', linewidth=1.5, label="Downbeat (t=0)")
    # subdivision lines
    for j in range(-nn, nn+1):
        if j!=0:
            ax.axvline(j*subdiv_len, color='gray', linestyle=':', alpha=0.5)

    sm = plt.cm.ScalarMappable(cmap=cmap, norm=plt.Normalize(W_start, W_end))
    sm.set_array([])
    cbar = plt.colorbar(sm, ax=ax)
    cbar.set_label('Time in recording (s)')

    custom = [
        Line2D([0],[0],color='blue',linestyle='-', lw=2),
        Line2D([0],[0],marker='o', color='w', markerfacecolor='blue', ms=8, markeredgecolor='k'),
        Line2D([0],[0],color='blue',linestyle='--', lw=2),
        Line2D([0],[0],marker='x', color='w', markeredgecolor='blue', ms=8),
        Line2D([0],[0],color='k', lw=2)
    ]
    labels = ["Left Trajectory","Left Onset","Right Trajectory",
              "Right Onset","Downbeat (t=0)"]
    ax.legend(custom, labels, loc='upper left', framealpha=0.3)

    ax.set_xlabel("Time relative to downbeat (s)")
    ax.set_ylabel("Foot Y Position")
    ax.set_title(
        f"Foot Trajectories ±{2*nn/n_subdiv_per_beat/ n_beats_per_cycle:.1f} beats around downbeats\n"
        f"{file_name} | window {W_start}-{W_end}s | {mode}",
        fontsize=10
    )
    ax.grid(True, alpha=0.3)
    plt.tight_layout()

    return fig, ax


In [3]:
# mode_csv = "data/subset_dance_annotation/BKO_E1_D5_04_Suku_Dancers.csv"

mode_csv_list = os.listdir("data/subset_dance_annotation")

for mode_csv in mode_csv_list:
    file_name = mode_csv.split("_Dancers")[0]
    mode_df = pd.read_csv("data/subset_dance_annotation/" + mode_csv)

    mode_group = mode_df[mode_df["mocap"] == "gr"].reset_index(drop=True)
    mode_individual = mode_df[mode_df["mocap"] == "in"].reset_index(drop=True)
    mode_audience = mode_df[mode_df["mocap"] == "au"].reset_index(drop=True)

    # helper to extract a (start, end) tuple or None
    def get_segment(df, name):
        if df.empty:
            print(f"⚠️  No rows for mode '{name}', skipping.")
            return None
        return (df["Start (in sec)"].iat[0],
                df["End (in sec)"].iat[0])

    # build a dict of segments
    segments = {
        "group":      get_segment(mode_group,      "gr"),
        "individual": get_segment(mode_individual, "in"),
        "audience":   get_segment(mode_audience,   "au")
    }

    # filter out the empty ones
    tsegment = {mode: seg for mode, seg in segments.items() if seg is not None}

    save_dir = f"output_static_plot/foot_trajectories/{file_name}"
    os.makedirs(save_dir, exist_ok=True)
    for mode, tup in tsegment.items():
        fig, ax = plot_foot_trajectories(
            file_name= file_name,
            mode = mode,
            base_path_cycles="data/virtual_cycles",
            base_path_logs="data/logs_v1_may",
            W_start=tup[0], W_end=tup[1],
            n_beats_per_cycle=4, n_subdiv_per_beat=12, nn=8
        )
        fig.savefig(os.path.join(save_dir, f"{file_name}_{tup[0]}_{tup[1]}.png"))
        plt.close(fig)
        # plt.show()


⚠️  No rows for mode 'gr', skipping.
⚠️  No rows for mode 'au', skipping.
⚠️  No rows for mode 'gr', skipping.
⚠️  No rows for mode 'au', skipping.
⚠️  No rows for mode 'gr', skipping.
⚠️  No rows for mode 'gr', skipping.
⚠️  No rows for mode 'au', skipping.
⚠️  No rows for mode 'gr', skipping.
⚠️  No rows for mode 'au', skipping.
⚠️  No rows for mode 'gr', skipping.
⚠️  No rows for mode 'au', skipping.
⚠️  No rows for mode 'gr', skipping.
⚠️  No rows for mode 'au', skipping.
⚠️  No rows for mode 'gr', skipping.
⚠️  No rows for mode 'au', skipping.
⚠️  No rows for mode 'gr', skipping.
⚠️  No rows for mode 'au', skipping.
⚠️  No rows for mode 'gr', skipping.
⚠️  No rows for mode 'au', skipping.
⚠️  No rows for mode 'gr', skipping.
⚠️  No rows for mode 'in', skipping.


In [ ]:

fig, ax = plot_foot_trajectories(
    file_name="BKO_E1_D2_04_Maraka",
    base_path_cycles="data/virtual_cycles",
    base_path_logs="data/logs_v1_may",
    W_start= 50, W_end= 100,
    n_beats_per_cycle=4, n_subdiv_per_beat=12, nn=8
)
plt.show()


